# Download data

In [1]:
import yfinance as yf

SYMBOL = "AAPL"
HISTORY = "10y "

all_day_k = yf.Ticker(SYMBOL).history(period = HISTORY , interval = "1d")

In [2]:
all_day_k

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-07-22 00:00:00-04:00,13.316285,13.325277,13.192567,13.218613,207796400,0.0,0.0
2013-07-23 00:00:00-04:00,13.208998,13.238765,12.982957,12.991639,369395600,0.0,0.0
2013-07-24 00:00:00-04:00,13.609919,13.785419,13.496124,13.658911,591936800,0.0,0.0
2013-07-25 00:00:00-04:00,13.664804,13.686509,13.513179,13.596588,229493600,0.0,0.0
2013-07-26 00:00:00-04:00,13.497365,13.675346,13.467599,13.673796,200152400,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-13 00:00:00-04:00,190.500000,191.190002,189.779999,190.539993,41342300,0.0,0.0
2023-07-14 00:00:00-04:00,190.229996,191.179993,189.630005,190.690002,41573900,0.0,0.0
2023-07-17 00:00:00-04:00,191.899994,194.320007,191.809998,193.990005,50520200,0.0,0.0


In [3]:
#all_day_k = all_day_k.drop(columns=["Dividends","Stock Splits"])

all_day_k = all_day_k[ :-1]

In [4]:
all_day_k

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-07-22 00:00:00-04:00,13.316285,13.325277,13.192567,13.218613,207796400,0.0,0.0
2013-07-23 00:00:00-04:00,13.208998,13.238765,12.982957,12.991639,369395600,0.0,0.0
2013-07-24 00:00:00-04:00,13.609919,13.785419,13.496124,13.658911,591936800,0.0,0.0
2013-07-25 00:00:00-04:00,13.664804,13.686509,13.513179,13.596588,229493600,0.0,0.0
2013-07-26 00:00:00-04:00,13.497365,13.675346,13.467599,13.673796,200152400,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-12 00:00:00-04:00,189.679993,191.699997,188.470001,189.770004,60750200,0.0,0.0
2023-07-13 00:00:00-04:00,190.500000,191.190002,189.779999,190.539993,41342300,0.0,0.0
2023-07-14 00:00:00-04:00,190.229996,191.179993,189.630005,190.690002,41573900,0.0,0.0


# Construct dataset 製作切合AI模型

In [5]:
import numpy as np
import pandas as pd

PAST_WIN_LEN = 100
CLASSES = [ "Bull" , "Bear"  ]  
LABEL_BULL = CLASSES.index( "Bull" )
LABEL_BEAR = CLASSES.index( "Bear" )

x,y = [ ] , [ ]
for today_i in range (len(all_day_k)):
    # get day-k in the past 100-day window and the forward 1-day window
    day_k_past = all_day_k[:today_i+1]
    day_k_forward = all_day_k [today_i+1:]
    if len (day_k_past) < PAST_WIN_LEN or len(day_k_forward) < 1:
        continue
    day_k_past_win = day_k_past[-PAST_WIN_LEN:]
    day_k_forward_win = day_k_forward[:1]
    
    # find label
    today_price = day_k_past_win.iloc[-1]["Close"]
    tomorrow_price = day_k_forward_win.iloc[0]["Close"]
    label = LABEL_BULL if tomorrow_price > today_price else LABEL_BEAR
    
    # Store 
    x.append(day_k_past_win.values)
    y.append(label)
    
x,y = np.array(x), np.array(y)
    

In [6]:
x.shape  #第一維度表示多少筆數據 第二維度每筆一百天 第三表示每天有五個數字

(2415, 100, 7)

In [7]:
y.shape #只有一維度 表示共有多少筆數據

(2415,)

In [8]:
y  # 0 is bull 1 is bear

array([1, 1, 1, ..., 0, 0, 1])

# Split dataset to training/validation/test dataset 
將數據分為 訓練 驗證 測試    
驗證模駔 主要是用來防止訓練模組時發生的OVER FITTING
TEST 用來評估 完成訓練後的模型的性能


In [9]:
TRAIN_SPLIT, VAL_SPLIT, TEST_SPLIT = 0.7,0.2,0.1

#Take the last portion to be the test dataset
test_split_index = - round(len(x) * TEST_SPLIT)
x_other, x_test = np.split (x, [test_split_index])
y_other, y_test = np.split (y, [test_split_index])

# shuffle the training portion and split into training and validation datasets
train_split_index = round(len(x) * TRAIN_SPLIT)
indexes = np.arange(len(x_other))
np.random.shuffle(indexes)
train_indexes,val_indexes = np.split(indexes,[train_split_index])
x_train, x_val = x_other[train_indexes], x_other[val_indexes]
y_train, y_val = y_other[train_indexes], y_other[val_indexes]
 

In [10]:
# show label distribution  
label_distribution = pd.DataFrame([{"Dataset" : "train",
                                                         "Bull":np.count_nonzero(y_train == LABEL_BULL),
                                                         "Bear": np.count_nonzero(y_train == LABEL_BEAR)},
                                                         {"Dataset" : "val",
                                                         "Bull":np.count_nonzero(y_val == LABEL_BULL),
                                                         "Bear": np.count_nonzero(y_val == LABEL_BEAR)},
                                                         {"Dataset" : "test",
                                                         "Bull":np.count_nonzero(y_test == LABEL_BULL),
                                                         "Bear": np.count_nonzero(y_test == LABEL_BEAR)}])
label_distribution

,Dataset,Bull,Bear
0,train,891,799
1,val,260,223
2,test,122,120


In [11]:
# Balance label of teset dataset
x_test_bull = x_test [  y_test == LABEL_BULL]
x_test_bear = x_test [ y_test == LABEL_BEAR]

min_n_labels = min(len(x_test_bull), len(x_test_bear))

x_test_bull = x_test_bull [np.random.choice(len(x_test_bull), min_n_labels, replace= False ),:]
x_test_bear = x_test_bear [np.random.choice(len(x_test_bull), min_n_labels, replace= False ),:]
x_test = np.vstack([ x_test_bull, x_test_bear ])

y_test = np.array([LABEL_BULL] * min_n_labels + [ LABEL_BEAR ] * min_n_labels )

# test datasets label distribution
pd.DataFrame( [ {"Dataset" : "test",
                            "Bull":np.count_nonzero(y_test == LABEL_BULL),
                            "Bear": np.count_nonzero(y_test == LABEL_BEAR) } ] )

,Dataset,Bull,Bear
0,test,120,120


# Save datasets

In [12]:
np.savez("datasets.npz",x_train = x_train , y_train=y_train,x_val = x_val, y_val=y_val, x_test=x_test,y_test=y_test)